In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import pystan

from read_data import df


In [ ]:
df['DEATH_EVENT'].value_counts()

In [ ]:
na_counts = df.isna().sum()
na_counts

In [ ]:
df

In [ ]:
smoke = df[df['smoking'] == 1]
no_smoke = df[df['smoking'] == 0]
male = df[df['sex'] == 1]
female = df[df['sex'] == 0]

In [ ]:
female

In [ ]:
is_binary = df.isin([0,1]).all().values
bin_names = df.columns[is_binary].tolist()
quant_names = df.columns[~is_binary].tolist()

In [ ]:
df_binary = df[bin_names]

# Calculating the correlation matrix for binary variables
corr_matrix = df_binary.corr()

# Plotting the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Matrix of Binary Variables")
plt.show()

From this correlation matrix we can see that there is a relationship between sex and smoking, with female composed of the majority of non-smoker. This mean we need to further handling this dependency while we build the model.

In [ ]:
df_quant = df[quant_names]
corr_matrix_cont = df_quant.corr()

# Plotting the correlation matrix for continuous variables
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix_cont, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Matrix of Continuous Variables")
plt.show()

In [ ]:
y = df['DEATH_EVENT']
X = df.drop(columns = {'DEATH_EVENT', 'time'})

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_train_sm = sm.add_constant(X_train_scaled)
X_test_sm = sm.add_constant(X_test_scaled)

# Fitting the logistic regression model
logit_model = sm.Logit(y_train, X_train_sm).fit()

# Extracting the summary which includes p-values among other statistics
model_summary = logit_model.summary()

y_pred = logit_model.predict(X_test_sm)
# accuracy = accuracy_score(y_pred, y_test)

# print(f"Accuracy: {accuracy}")

model_summary

In [ ]:
stan_model_code = """
data {
    int<lower=0> N; // Number of data points
    int<lower=0,upper=1> y[N]; // Outcome variable (0 or 1)
    vector[N] x; // Predictor variable
}
parameters {
    real alpha; // Intercept
    real beta; // Slope
}
model {
    // Priors
    alpha ~ normal(0, 10);
    beta ~ normal(0, 10);

    // Likelihood
    for (i in 1:N)
        y[i] ~ bernoulli_logit(alpha + beta * x[i]);
}
"""

# Data for Stan model
data = {
    'N': len(y),
    'y': y,
    'x': X['age'],
}

# Compile and fit the model
stan_model = pystan.StanModel(model_code=stan_model_code)
# fit = stan_model.sampling(data=data, iter=3000, chains=4, warmup=1000, seed=101)

# # Extract and print summary of the model fit
# fit_summary = fit.summary()
# print(fit_summary)

In [ ]:
fit = stan_model.sampling(data=data, iter=3000, chains=4, warmup=1000, seed=101)
